<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/01.Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# For Colab to use Tensorflow 2.X
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
import tensorflow as tf

# One Feature Node

In [5]:
class One_Feature_Node:
    def __init__(self):
        self.w = tf.Variable([[0.1]]) # variable (변수: 변하는 값) w:weight 
        self.b = tf.Variable([[0.5]]) # (변수, 파라미터) b:bias(편향)  변수는 업데이트 되면서 바뀌는 값임 
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        out = tf.matmul(x,self.w)    # 인풋에다 가중치 곱해줌
        print("matmul result: ", out.numpy())
        out = tf.add(out, self.b)   # 이후 편향값 더해줌
        print("adding bias result: ", out.numpy())
        out = tf.math.sigmoid(out)  # 마지막으로 활성함수 취해줌
        print("sigmoid result:", out.numpy())
        return out

In [6]:
# Single Input
x = tf.constant([[1.0]])  # 인풋값은 변하는 값이 아니므로 constant(상수) 고정값으로 넣어줌

one_feature_node = One_Feature_Node()
one_feature_node(x)

matmul result:  [[0.1]]
adding bias result:  [[0.6]]
sigmoid result: [[0.6456563]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.6456563]], dtype=float32)>

# Two Features Node

In [7]:
class Two_Features_Node:
    def __init__(self):
        self.w = tf.Variable([[0.1], [0.2]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        out = tf.matmul(x,self.w)
        print("matmul result: ", out.numpy())
        out = tf.add(out, self.b)
        print("adding bias result: ", out.numpy())
        out = tf.math.sigmoid(out)
        print("sigmoid result:", out.numpy())
        return out

In [8]:
# Multi Features Single Input (Two features for one data)
x = tf.constant([[1.0,2.0]])

two_features_node = Two_Features_Node()
two_features_node(x)

matmul result:  [[0.5]]
adding bias result:  [[1.]]
sigmoid result: [[0.73105854]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.73105854]], dtype=float32)>

# N inputs for Two Features Node

In [9]:
# Multi Input Data (3 samples)
x = tf.constant([[1.0,2.0],[2.0,3.0],[4.0,5.0]])

two_features_node = Two_Features_Node()
two_features_node(x)

matmul result:  [[0.5]
 [0.8]
 [1.4]]
adding bias result:  [[1. ]
 [1.3]
 [1.9]]
sigmoid result: [[0.73105854]
 [0.785835  ]
 [0.8698916 ]]


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.73105854],
       [0.785835  ],
       [0.8698916 ]], dtype=float32)>

# tf.function speed check
tf.function decoration put your python onto autograph.  
This enables faster execution, running on GPU or TPU, or exporting to SavedModel.

In [10]:
import timeit

In [11]:
class Two_Features_Node2:
    def __init__(self):
        self.w = tf.Variable([[0.1], [0.2]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        for i in range(50):
            out = tf.matmul(x,self.w)
            out = tf.add(out, self.b)
            out = tf.math.sigmoid(out)
        return out
    
    @tf.function
    def get_output_faster(self,x):
        for i in range(50):
            out = tf.matmul(x,self.w)
            out = tf.add(out, self.b)
            out = tf.math.sigmoid(out)
        return out

In [12]:
two_features_node2 = Two_Features_Node2()
get_output_without_tf_function_timeit = timeit.timeit(lambda: two_features_node2.get_output(x), number=300)
get_output_faster_with_tf_function_timeit = timeit.timeit(lambda: two_features_node2.get_output_faster(x), number=300)
time_diff = round(get_output_without_tf_function_timeit / get_output_faster_with_tf_function_timeit, 1)

In [13]:
print("get_output_faster(@tf.function) is " + str(time_diff) + " times faster than get_output_without_tf_function!")

get_output_faster(@tf.function) is 10.4 times faster than get_output_without_tf_function!
